# Análise com SQL Avançado
## U.S. EPA Food Commodity Intake Database (FCID)
### [https://fcid.foodrisk.org/](https://fcid.foodrisk.org/)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

# Importando Tabelas do FCID

In [2]:
DROP TABLE IF EXISTS Crop_Group;
DROP TABLE IF EXISTS FCID_Description;
DROP TABLE IF EXISTS Recipes;
DROP TABLE IF EXISTS Intake;

CREATE TABLE Crop_Group (
  CGN VARCHAR(2),
  CGL VARCHAR(6),
  Crop_Group_Description VARCHAR(80),
  PRIMARY KEY (CGL)
) AS SELECT
  CGN, CGL, Crop_Group_Description
FROM CSVREAD('../../data/food-intake/basics/FCID_Cropgroup_Description.csv');

CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code),
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('../../data/food-intake/basics/FCID_Code_Description.csv');

CREATE TABLE Recipes (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(5, 2),
  CSFII_9498_IND TINYINT,
  WWEIA_9904_IND TINYINT,
  WWEIA_0510_IND TINYINT,
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight, CSFII_9498_IND, WWEIA_9904_IND, WWEIA_0510_IND
FROM CSVREAD('../../data/food-intake/recipes/Recipes_WWEIA_FCID_0510.csv');

CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(13,7),
  Intake_BW DECIMAL(13,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('../../data/food-intake/consumption/Commodity_CSFFM_Intake_0510-cropped.csv');

# Visualizando as Tabelas

In [3]:
SELECT * FROM Crop_Group LIMIT 10;

In [4]:
SELECT * FROM FCID_Description LIMIT 10;

In [5]:
SELECT * FROM Recipes LIMIT 10;

In [6]:
SELECT * FROM Intake LIMIT 10;

# Métricas

Considere que a tabela Intake registra alimentos consumidos por 1.489 pessoas. Considere as seguintes métricas para um alimento:

| Métrica | Descrição |
| --- | --- |
| Popularidade | número de pessoas (dentre as 1.489) que consumiram o alimento |
| Intake_Sum | total consumido do alimento pelas 1.489 pessoas em gramas |
| Intake_AVG | média de consumo do alimento em gramas |
| Intake_AVG_BW | média de consumo do alimento x peso da pessoa |
| Recipes | número de receitas (dentre as 7.154 receitas) que têm o produto como ingrediente |

## 1)  Construa uma View que apresente essas métricas por produto

* Veja exemplo em: `/data/food-intake/computed/commodity-profile.csv`
* Importante: esta tabela foi feita com um número maior de registros, portanto os valores não serão iguais aos seus

In [7]:
CREATE VIEW Metrics AS(SELECT I.FCID_CODE, COUNT(*) AS Popularidade, SUM(I.Intake) as Intake_Sum, AVG(I.Intake) AS Intake_AVG, AVG(I.Intake_BW) AS Intake_AVG_BW, R.Total_Receita AS Recipes
FROM Intake as I 
LEFT JOIN (SELECT FCID_CODE, COUNT(*) AS Total_Receita
            FROM Recipes
            GrOUP BY FCID_CODE) as R ON R.FCID_CODE = I.FCID_CODE
GROUP BY I.FCID_CODE)

## 2) Como você analisaria a correlação entre as métricas?

* Por exemplo, produtos mais populares são mais consumidos (em número de pessoas ou em quantidade)?
* Proponha uma ou mais queries para fazer esta análise

In [9]:
CREATE VIEW Correlacao AS
SELECT
    'Popularidade' AS "Coluna 1",
    'Intake_sum' AS "Coluna 2",
    (SUM(Popularidade * Intake_Sum) - SUM(Popularidade) * SUM(Intake_Sum) / COUNT(*)) /
    (SQRT((SUM(Popularidade * Popularidade) - SUM(Popularidade) * SUM(Popularidade) / COUNT(*)) *
          (SUM(Intake_Sum * Intake_Sum) - SUM(Intake_Sum) * SUM(Intake_Sum) / COUNT(*)))) AS Pearson_1X2
FROM Metrics
UNION ALL
SELECT
    'Popularidade' AS "Coluna 1",
    'Intake_avg' AS "Coluna 2",
    (SUM(Popularidade * Intake_Avg) - SUM(Popularidade) * SUM(Intake_Avg) / COUNT(*)) /
    (SQRT((SUM(Popularidade * Popularidade) - SUM(Popularidade) * SUM(Popularidade) / COUNT(*)) *
          (SUM(Intake_Avg * Intake_Avg) - SUM(Intake_Avg) * SUM(Intake_Avg) / COUNT(*)))) AS Pearson_1X2
FROM Metrics
UNION ALL
SELECT
    'Popularidade' AS "Coluna 1",
    'Intake_avg_bw' AS "Coluna 2",
    (SUM(Popularidade * Intake_avg_bw) - SUM(Popularidade) * SUM(Intake_avg_bw) / COUNT(*)) /
    (SQRT((SUM(Popularidade * Popularidade) - SUM(Popularidade) * SUM(Popularidade) / COUNT(*)) *
          (SUM(Intake_avg_bw * Intake_avg_bw) - SUM(Intake_avg_bw) * SUM(Intake_avg_bw) / COUNT(*)))) AS Pearson_1X2
FROM Metrics
UNION ALL
SELECT
    'Popularidade' AS "Coluna 1",
    'Recipes' AS "Coluna 2",
    (SUM(Popularidade * Recipes) - SUM(Popularidade) * SUM(Recipes) / COUNT(*)) /
    (SQRT((SUM(Popularidade * Popularidade) - SUM(Popularidade) * SUM(Popularidade) / COUNT(*)) *
          (SUM(Recipes * Recipes) - SUM(Recipes) * SUM(Recipes) / COUNT(*)))) AS Pearson_1X2
FROM Metrics
UNION ALL
SELECT
    'Intake_sum' AS "Coluna 1",
    'Intake_avg' AS "Coluna 2",
    (SUM(Intake_sum * Intake_avg) - SUM(Intake_sum) * SUM(Intake_avg) / COUNT(*)) /
    (SQRT((SUM(Intake_sum * Intake_sum) - SUM(Intake_sum) * SUM(Intake_sum) / COUNT(*)) *
          (SUM(Intake_avg * Intake_avg) - SUM(Intake_avg) * SUM(Intake_avg) / COUNT(*)))) AS Pearson_1X2
FROM Metrics
UNION ALL
SELECT
    'Intake_sum' AS "Coluna 1",
    'Intake_avg_bw' AS "Coluna 2",
    (SUM(Intake_sum * Intake_avg_bw) - SUM(Intake_sum) * SUM(Intake_avg_bw) / COUNT(*)) /
    (SQRT((SUM(Intake_sum * Intake_sum) - SUM(Intake_sum) * SUM(Intake_sum) / COUNT(*)) *
          (SUM(Intake_avg_bw * Intake_avg_bw) - SUM(Intake_avg_bw) * SUM(Intake_avg_bw) / COUNT(*)))) AS Pearson_1X2
FROM Metrics
UNION ALL
SELECT
    'Intake_avg' AS "Coluna 1",
    'Intake_avg_bw' AS "Coluna 2",
    (SUM(Intake_avg * Intake_avg_bw) - SUM(Intake_avg) * SUM(Intake_avg_bw) / COUNT(*)) /
    (SQRT((SUM(Intake_avg * Intake_avg) - SUM(Intake_avg) * SUM(Intake_avg) / COUNT(*)) *
          (SUM(Intake_avg_bw * Intake_avg_bw) - SUM(Intake_avg_bw) * SUM(Intake_avg_bw) / COUNT(*)))) AS Pearson_1X2
FROM Metrics
UNION ALL
SELECT
    'Intake_sum' AS "Coluna 1",
    'Recipes' AS "Coluna 2",
    (SUM(Intake_sum * Recipes) - SUM(Intake_sum) * SUM(Recipes) / COUNT(*)) /
    (SQRT((SUM(Intake_sum * Intake_sum) - SUM(Intake_sum) * SUM(Intake_sum) / COUNT(*)) *
          (SUM(Recipes * Recipes) - SUM(Recipes) * SUM(Recipes) / COUNT(*)))) AS Pearson_1X2
FROM Metrics
UNION ALL
SELECT
    'Intake_avg' AS "Coluna 1",
    'Recipes' AS "Coluna 2",
    (SUM(Intake_avg * Recipes) - SUM(Intake_avg) * SUM(Recipes) / COUNT(*)) /
    (SQRT((SUM(Intake_avg * Intake_avg) - SUM(Intake_avg) * SUM(Intake_avg) / COUNT(*)) *
          (SUM(Recipes * Recipes) - SUM(Recipes) * SUM(Recipes) / COUNT(*)))) AS Pearson_1X2
FROM Metrics
UNION ALL
SELECT
    'Intake_avg_bw' AS "Coluna 1",
    'Recipes' AS "Coluna 2",
    (SUM(Intake_avg_bw * Recipes) - SUM(Intake_avg_bw) * SUM(Recipes) / COUNT(*)) /
    (SQRT((SUM(Intake_avg_bw * Intake_avg_bw) - SUM(Intake_avg_bw) * SUM(Intake_avg_bw) / COUNT(*)) *
          (SUM(Recipes * Recipes) - SUM(Recipes) * SUM(Recipes) / COUNT(*)))) AS Pearson_1X2
FROM Metrics;


## 3) Podemos criar grupos de consumidores conforme um perfil?
* por exemplo, consumidores podem ser agrupados por alimentos que comem predominantemente?
* como você associaria grupos a classes?

## 4) Que métricas podem ser analisadas para a comparação de perfis?
* escreva uma query SQL que calcule pelo menos uma métrica comparativa